In [1]:
import numpy as np
import pandas as pd
import os

## Applying various tactics for class imbalance

In [2]:
processed_data_path = os.path.join(os.path.pardir,'data','processed')
train_file_path = os.path.join(processed_data_path, 'train1.csv')
train_df = pd.read_csv(train_file_path, index_col='encounter_id')
# df.head(30)
test_file_path = os.path.join(processed_data_path, 'test1.csv')
test_df = pd.read_csv(test_file_path, index_col='encounter_id')
test_df.shape
df = train_df
df.head()
# test_df.isnull().values.any()

,hospital_death,hospital_id,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,...,icu_type_CCU-CTICU,icu_type_CSICU,icu_type_CTICU,icu_type_Cardiac ICU,icu_type_MICU,icu_type_Med-Surg ICU,icu_type_Neuro ICU,icu_type_SICU,gender_F,gender_M
encounter_id,,,,,,,,,,,,,,,,,,,,,
66154,0,118,68.0,22.730000,0,180.3,92,0.541667,0,73.9,...,0,0,1,0,0,0,0,0,0,1
114252,0,81,77.0,27.420000,0,160.0,90,0.927778,0,70.2,...,0,0,0,0,0,1,0,0,1,0
119783,0,118,25.0,31.950000,0,172.7,93,0.000694,0,95.3,...,0,0,0,0,0,1,0,0,1,0
79267,0,118,81.0,22.640000,1,165.1,92,0.000694,0,61.7,...,0,0,1,0,0,0,0,0,1,0
92056,0,33,19.0,14.844926,0,188.0,91,0.073611,0,68.0,...,0,0,0,0,0,1,0,0,0,1


## using upsampling
## using down sampling
## split train and test
## using logistic regression with up and down sampling
## change performance metric
## support vector Machine




In [3]:
df_majority = df[df['hospital_death']==0]
df_minority = df[df['hospital_death']==1]
# df_majority.shape
# (83798, 235)
# df_minority.shape
# (7915, 235)

In [4]:
from sklearn.utils import resample

In [5]:
df_minority_upsampled = resample(df_minority, replace=True, n_samples=83798, random_state=123)

In [6]:
df_upsampled = pd.concat([df_majority, df_minority_upsampled])

In [7]:
df_upsampled['hospital_death'].value_counts()

1    83798
0    83798
Name: hospital_death, dtype: int64

In [8]:
# verifiying  class is balanced 
num_obs = len(df_upsampled)
num_true = len(df_upsampled.loc[df_upsampled['hospital_death'] == 1])
num_false = len(df_upsampled.loc[df_upsampled['hospital_death'] == 0])
print("Number of True cases:  {0} ({1:2.2f}%)".format(num_true, (num_true/num_obs) * 100))
print("Number of False cases: {0} ({1:2.2f}%)".format(num_false, (num_false/num_obs) * 100))

Number of True cases:  83798 (50.00%)
Number of False cases: 83798 (50.00%)


In [9]:
X = df_upsampled.loc[:,'hospital_id':].values.astype('float')
y= df_upsampled['hospital_death'].ravel()


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 
                            
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(134076, 234) (134076,)
(33520, 234) (33520,)


In [30]:
## after split make sure class ration is main tained

print("Original True  : {0} ({1:0.3f}%)".format(len(df_upsampled.loc[df['hospital_death'] == 1]), (len(df_upsampled.loc[df['hospital_death'] == 1])/len(df_upsampled.index)) * 100.0))
print("Original False : {0} ({1:0.2f}%)".format(len(df_upsampled.loc[df['hospital_death'] == 0]), (len(df_upsampled.loc[df['hospital_death'] == 0])/len(df_upsampled.index)) * 100.0))
print("")
print("Training True  : {0} ({1:0.2f}%)".format(len(y_train[y_train[:] == 1]), (len(y_train[y_train[:] == 1])/len(y_train) * 100.0)))
print("Training False : {0} ({1:0.2f}%)".format(len(y_train[y_train[:] == 0]), (len(y_train[y_train[:] == 0])/len(y_train) * 100.0)))
print("")
print("Test True      : {0} ({1:0.2f}%)".format(len(y_test[y_test[:] == 1]), (len(y_test[y_test[:] == 1])/len(y_test) * 100.0)))
print("Test False     : {0} ({1:0.2f}%)".format(len(y_test[y_test[:] == 0]), (len(y_test[y_test[:] == 0])/len(y_test) * 100.0)))


Original True  : 83798 (50.000%)
Original False : 83798 (50.00%)

Training True  : 66836 (49.85%)
Training False : 67240 (50.15%)

Test True      : 16962 (50.60%)
Test False     : 16558 (49.40%)


## Logistic Regression Model

In [11]:
from sklearn.linear_model import LogisticRegression

In [32]:
# create model
model_lr_2 = LogisticRegression(random_state=0)

In [34]:
model_lr_2.fit(X_train,y_train)

/Users/m785017/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

## peformance metrics

In [35]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score,roc_auc_score


In [ ]:
# Predict on training set
pred_y_2 = model_lr_2.predict(X)

In [37]:
# performance metrics
# accuracy
print ('accuracy for logistic regression - version 1 : {0:.2f}'.format(accuracy_score(y_test, model_lr_2.predict(X_test))))
# confusion matrix
print ('confusion matrix for logistic regression - version 1: \n {0}'.format(confusion_matrix(y_test, model_lr_2.predict(X_test))))
# precision 
print ('precision for logistic regression - version 1 : {0:.2f}'.format(precision_score(y_test, model_lr_2.predict(X_test))))
# precision 
print ('recall for logistic regression - version 1 : {0:.2f}'.format(recall_score(y_test, model_lr_2.predict(X_test))))
# ROC score 
print ('roc score for logistic regression - version 1 : {0:.2f}'.format(roc_auc_score(y_test, model_lr_2.predict(X_test))))


accuracy for logistic regression - version 1 : 0.81
confusion matrix for logistic regression - version 1: 
 [[13574  2984]
 [ 3496 13466]]
precision for logistic regression - version 1 : 0.82
recall for logistic regression - version 1 : 0.79
roc score for logistic regression - version 1 : 0.81


In [39]:
test = test_df 

In [38]:
def get_submission_file(model, filename):
    #converting to matrix
    test_X = test.values.astype('float')
    #make predictions
    predictions = model.predict(test_X)
    # submission data Frame
    df_submission = pd.DataFrame({'encounter_id':test.index, 'hospital_death':predictions})
    # submission file
    submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
    submission_file_path = os.path.join(submission_data_path, filename)
    #write to the file
    df_submission.to_csv(submission_file_path, index=False)

In [40]:
get_submission_file(model_lr_2, 'lr_upsampled.csv')

## Down sampling technique

In [41]:
df_majority = df[df['hospital_death']==0]
df_minority = df[df['hospital_death']==1]
# df_majority.shape
# (83798, 235)
# df_minority.shape
# (7915, 235)

In [42]:
df_majority_downsampled = resample(df_majority, replace=True, n_samples=7915, random_state=123)

In [46]:
df_downsampled = pd.concat([df_minority, df_majority_downsampled])
df_downsampled.shape
# (15830, 235)

(15830, 235)

In [48]:
X = df_downsampled.loc[:,'hospital_id':].values.astype('float')
y= df_downsampled['hospital_death'].ravel()


In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 
                            
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(12664, 234) (12664,)
(3166, 234) (3166,)


In [50]:
model_lr_3 = LogisticRegression(random_state=0)

In [51]:
model_lr_3.fit(X_train,y_train)

/Users/m785017/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=0, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [53]:
# performance metrics
# accuracy
print ('accuracy for logistic regression - version 1 : {0:.2f}'.format(accuracy_score(y_test, model_lr_3.predict(X_test))))
# confusion matrix
print ('confusion matrix for logistic regression - version 1: \n {0}'.format(confusion_matrix(y_test, model_lr_3.predict(X_test))))
# precision 
print ('precision for logistic regression - version 1 : {0:.2f}'.format(precision_score(y_test, model_lr_3.predict(X_test))))
# precision 
print ('recall for logistic regression - version 1 : {0:.2f}'.format(recall_score(y_test, model_lr_3.predict(X_test))))
# ROC score 
print ('roc score for logistic regression - version 1 : {0:.2f}'.format(roc_auc_score(y_test, model_lr_3.predict(X_test))))


accuracy for logistic regression - version 1 : 0.79
confusion matrix for logistic regression - version 1: 
 [[1302  296]
 [ 359 1209]]
precision for logistic regression - version 1 : 0.80
recall for logistic regression - version 1 : 0.77
roc score for logistic regression - version 1 : 0.79


In [54]:
get_submission_file(model_lr_3, 'lr_downsampled.csv')

## support vector Machine

In [12]:
from sklearn.svm import SVC

In [13]:
train_df = pd.read_csv(train_file_path, index_col='encounter_id')
test_df = pd.read_csv(test_file_path, index_col='encounter_id')

In [14]:
X = train_df.loc[:,'hospital_id':].values.astype('float')
y= train_df['hospital_death'].ravel()

In [15]:
svm_1 = SVC(kernel="linear", class_weight="balanced", probability=True)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 

In [ ]:
svm_1.fit(X_train, y_train)

In [ ]:
# performance metrics
# accuracy
print ('accuracy for SVM - version 1 : {0:.3f}'.format(accuracy_score(y_test, svm_1.predict(X_test))))
# confusion matrix
print ('confusion matrix for SVM - version 1: \n {0}'.format(confusion_matrix(y_test, svm_1.predict(X_test))))
# precision 
print ('precision for SVM - version 1 : {0:.23}'.format(precision_score(y_test, svm_1.predict(X_test))))
# precision 
print ('recall for SVM - version 1 : {0:.3f}'.format(recall_score(y_test, svm_1.predict(X_test))))
# ROC score 
print ('roc score for SVM - version 1 : {0:.3f}'.format(roc_auc_score(y_test, svm_1.predict(X_test))))


In [ ]:
get_submission_file(svm_1, 'svmachine.csv')